In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import tool
from crewai_tools import SerperDevTool
import os
from IPython.display import Markdown

In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = os.environ.get("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.environ.get("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_REGION_NAME"] = "us-east-1"

In [ ]:
# @tool("Tool Name")                           
# def my_simple_tool(question: str) -> str:           
#     """Tool description for clarity."""           
#     # Tool logic here                       
#     return "Tool output"                         

In [ ]:
@tool('SerperDevTool')
def search(search_query: str):
    """Search the web for information on a given topic"""
    tool = SerperDevTool(
    search_url="https://google.serper.dev/search",
    n_results=2,
)

    return(tool.run(search_query=search_query))

In [ ]:
@tool('CalculatorAdd')
def calculator_add(a,b):
    """Adds two numbers and gives a result"""
    return (a+b)

In [ ]:
modelId="bedrock/anthropic.claude-3-haiku-20240307-v1:0"

In [ ]:
# Configure the LLM
llm = LLM(model=modelId)

In [ ]:
# Define the Agent
router_agent = Agent(
    role='Router Assistant',
    goal='Assistant to route to the appropriate tool',
    backstory="You are an experienced assistant specializing in routing the flow to the appropriate tool.",
    verbose=False,
    allow_delegation=False,
    llm=llm,
    tools=[search, calculator_add]  
)

In [ ]:
# Define the Task
task = Task(
    description="Based on the user's questions: {questions}, route to the appropriate tool",
    expected_output="A properly well worded answer from the tool which can be used by the user.",
    agent=router_agent
)

In [ ]:
# Create the Crew
crew = Crew(
    agents=[router_agent],
    tasks=[task],
    verbose=False,
)

In [ ]:
# User input for travel preferences
user_input = {
    "questions": "What is 7+9"
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

In [ ]:
Markdown(result.raw)

In [ ]:
# Enabling Memory in the Agent
crew_with_memory = Crew(
    agents=[router_agent],
    tasks=[task],
    verbose=True,
    memory=True,  # Enable memory
    embedder={
        "provider": "aws_bedrock",
        "config": {
            "model": "amazon.titan-embed-text-v1",  # Embedding model for memory
            "vector_dimension": 1024
        }
    },
    
)

In [ ]:
# Executing the Crew with Memory
result_with_memory = crew_with_memory.kickoff(inputs=user_input)

In [ ]:
from IPython.display import Markdown

In [ ]:
Markdown(result_with_memory.raw)

In [ ]:
user_input = {
    "questions": "What is the result of the match between GT vs MI in IPL ?"
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

In [ ]:
Markdown(result.raw)

In [ ]:
user_input = {
    "questions": "Get the age of Aamir Khan of Bollywood and add 60 to it"
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

In [ ]:
Markdown(result.raw)